<a href="https://colab.research.google.com/github/MA-Barracas/insanely-fast-whisper-tutorial/blob/main/insanely_fast_whisper_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
# !pip install --upgrade pip
# !pip install --upgrade transformers datasets[audio] accelerate
# !pip install --upgrade pytubefix pydub

In [41]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset

In [44]:
from pytubefix import YouTube  # Cambiado a pytubefix
from pydub import AudioSegment
import os
import traceback  # Importar el módulo traceback

def descargar_audio_mp3(url):
    try:
        # Descarga del video
        yt = YouTube(url)
        video = yt.streams.filter(only_audio=True).first()
        output_file = video.download()

        # Conversión a MP3 usando pydub
        mp3_filename = os.path.splitext(output_file)[0] + ".mp3"
        audio = AudioSegment.from_file(output_file)
        audio.export(mp3_filename, format="mp3")

        # Eliminación del archivo original
        os.remove(output_file)

        print(f"Archivo MP3 guardado como: {mp3_filename}")
        return mp3_filename
    except Exception as e:
        print("Ha ocurrido un error: ", e)
        print(traceback.format_exc())
        return mp3_filename

# Ejemplo de uso
url = "https://www.youtube.com/watch?v=gqUQbjsYZLQ"
audi_filename = descargar_audio_mp3(url)

Archivo MP3 guardado como: /content/Cursor AI tutorial for beginners.mp3


In [45]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)


In [46]:
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
)

In [47]:
import os
os.listdir()

['.config',
 'y2mate.com - Cursor AI tutorial for beginners.mp3',
 'Cursor AI tutorial for beginners.mp3',
 'sample_data']

In [48]:
result = pipe(audi_filename)

/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:496: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


In [50]:
# len(result["text"])

In [51]:
# !pip install cohere

In [54]:
import cohere
from google.colab import userdata
co = cohere.Client(api_key=userdata.get('COHERE_API_KEY'))

query = f"""
Write a thorough summary for this text: '''
{result["text"]}
'''
Give extensive info about the content.
Discuss all important points raised throughout.
At the end, create a section of "Conclusions" and another for "Summary"
"""

cohere_query = co.chat(
  model="command-r-plus",
  message=query
)


In [55]:
from IPython.display import Markdown
display(Markdown(cohere_query.text))

## Text Summary

In this video, the host interviews Mike, a front-end developer who has been experimenting with Cursor AI. Mike shares his insights and best practices for getting the most out of Cursor AI and other similar tools. He emphasizes the importance of planning and having a developer mindset, even if one is not a developer. He suggests using tools like Figma or even just sketching ideas on paper before jumping into Cursor AI. He also introduces the concept of "measure twice, cut once," which involves spending time planning and visualizing the project before starting to code.

Mike then introduces the idea of using V0, a UI library, to create wireframes and mock-ups before moving to Cursor AI. He demonstrates how V0 can help visualize the final product and provide a foundation for Cursor AI to build upon. He also mentions cursor.directory, a website that provides pre-written prompts for different technologies, which can be copied and pasted into Cursor AI to improve its performance.

The discussion moves on to the importance of tagging documentation (docs) within Cursor AI. Mike explains that by providing the AI with the latest and most accurate information, users can improve the accuracy of the code generated by Cursor AI. He also suggests asking other AI models for help when Cursor AI gets stuck, providing them with the bug, the attempted solutions, and the expected output.

Mike highlights the benefits of using AI for explaining and teaching code, adding comments, and duplicating existing functionality with a twist. He emphasizes the value of providing context to AI models and suggests using templates or starter kits to speed up the development process. He also mentions his own free starter kit, which includes boilerplate code for common features like authentication and database integration.

The interview concludes with a discussion about the potential future of AI models and the importance of investing time in learning and preparing for their advancements. Mike and the host agree that the best time to be a developer is now, as the combination of coding skills and AI tools will lead to powerful results.

## Conclusions

This video offers valuable insights and best practices for anyone interested in using Cursor AI or similar AI tools for development. The key takeaways include the importance of planning, providing context, and utilizing additional resources such as V0, cursor.directory, and other AI models. By following these tips, users can improve the accuracy and efficiency of their AI-assisted development process.

## Summary

Mike, a front-end developer, shares his best practices for using Cursor AI effectively. He emphasizes planning, using tools like V0 for wireframing, and providing context to AI models. He introduces cursor.directory for pre-written prompts and suggests tagging docs within Cursor AI. Mike highlights the benefits of AI for explaining code and suggests asking other AI models for help when stuck. He also recommends using templates and starter kits to speed up development. The discussion concludes with thoughts on the future of AI models and the importance of investing time in learning now.

In [ ]:
from google.colab import runtime
runtime.restart_runtime()


In [62]:
# !pip install insanely-fast-whisper

In [63]:
# !pip install -q pipx && apt install python3.10-venv

In [1]:
!pipx run insanely-fast-whisper --file-name "Cursor AI tutorial for beginners.mp3"

⚠️  insanely-fast-whisper is already on your PATH and installed at
    /usr/local/bin/insanely-fast-whisper. Downloading and running anyway.
🤗 Transcribing... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0:00:07You have passed task=transcribe, but also have set `forced_decoder_ids` to [[1, None], [2, 50360]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of task=transcribe.
🤗 Transcribing... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0:00:12Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
🤗 Transcribing... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0:00:12The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
🤗 Tr

In [9]:
import json
text_ifw = json.load(open("output.json", "r"))

In [16]:
len(text_ifw["text"])

35661

In [18]:
import cohere
from google.colab import userdata
co = cohere.Client(api_key=userdata.get('COHERE_API_KEY'))
query = f"""
Write a thorough summary for this text: '''
{text_ifw["text"]}
'''
Give extensive info about the content.
Discuss all important points raised throughout.
At the end, create a section of "Conclusions" and another for "Summary"
"""

cohere_query = co.chat(
  model="command-r-plus",
  message=query
)

from IPython.display import Markdown
display(Markdown(cohere_query.text))

## Text Summary

In this video interview, the host invites Mike, a front-end developer, to share his best practices and strategies for using Cursor AI effectively. Mike begins by emphasizing the importance of planning and having a developer mindset. He suggests using tools like Figma or even simple sketches to visualize the desired outcome before prompting AI models. He also introduces the concept of "rubber ducking," where one explains their thoughts to a fictional duck, which helps with realizations and perspectives. 

Mike then introduces the use of V0, a platform that assists in visualizing the minimum viable product (MVP) of an app or website. He demonstrates how V0 can be used to create a clean-looking marketplace website, emphasizing its ability to provide a nice-looking UI with the Shatsian UI library. He suggests spending a good amount of time on V0, making at least 10-15 prompts to get the desired outcome before moving on to Cursor AI.

The discussion turns to cursor.directory, a website that provides prompts which can be copied and pasted into the Cursor codebase. These prompts ensure that Cursor has the necessary context and information about the technologies being used, such as Next.js. Mike demonstrates how to create a .cursor rules file in the root of a project and the benefits of providing this additional context to Cursor. He also mentions that if a specific technology is not listed on cursor.directory, one can prompt AI models like Cloud or ChatGVT to write similar prompts.

Mike emphasizes the importance of tagging documentation (docs) for the technologies being used. He demonstrates how to add the Next.js and Supabase docs to Cursor and explains that having access to the latest and most accurate information helps Cursor provide better solutions. He suggests that users should treat the AI models as new employees and provide them with thorough onboarding, including relevant documentation.

The host and Mike discuss the benefit of asking other AI models for help when Cursor gets stuck. Mike shares his strategy of providing the bug, the attempted solutions, and the expected outcome to another AI model, resulting in improved results. They also touch on the value of explaining code and teaching concepts using AI, as well as adding comments to code for better understanding.

Mike highlights the importance of duplicating existing functionality when making similar changes and providing context to AI models. He suggests using templates or starter kits that include boilerplate code for common features like authentication and database integration, emphasizing that Cursor and other AI platforms will likely provide more templates in the future. He offers his own free starter kit as an example.

The interview concludes with a discussion about building social media apps using Cursor and the host's previous video on the topic. They encourage viewers to use the comment section for questions, discussions, and sharing their own experiences with Cursor and AI development.

## Conclusions

The interview offers valuable insights and strategies for effectively using Cursor AI and similar tools. Planning, visualizing, and providing context are emphasized as key factors for successful outcomes. The host and Mike discuss the benefits of using V0 for initial planning and visualization, cursor.directory for providing technology-specific prompts, and tagging documentation for up-to-date information. Asking other AI models for help when Cursor gets stuck, explaining code, and using templates or starter kits are also highlighted as useful strategies.

## Summary

The host interviews Mike, a front-end developer, to gain insights into best practices for using Cursor AI. Mike emphasizes planning, visualization, and providing context to AI models. He introduces V0 for initial planning and cursor.directory for technology-specific prompts. Tagging documentation and asking other AI models for help when stuck are recommended strategies. Explaining code, adding comments, and using templates are also discussed. Mike offers his own free starter kit as an example. The interview concludes with a discussion about building social media apps and engaging viewers through the comment section.